In [ ]:
from google.colab import drive
import os

drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive/Colab Notebooks/core-analysis/")

%pip install -U segmentation-models
%pip install git+https://github.com/lucasb-eyer/pydensecrf.git

In [ ]:
import os
from os.path import join
from datetime import date
import pickle as pkl

os.environ["SM_FRAMEWORK"] = "tf.keras"

import cv2
import tensorflow as tf
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
from keras import callbacks
import segmentation_models as sm

from core_analysis.architecture import masked_loss
from core_analysis.preprocess import preprocess_batches
from core_analysis.postprocess import predict_tiles
from core_analysis.utils.tools import data_augmentation, adjust_rgb, undersample

print(tf.__version__)

In [ ]:
# Check the number of available GPUs.
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))
physical_devices = tf.config.list_physical_devices("GPU")
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
with open(
    join("data", "dataset", "dataset_forages_128x128_20230705.pickle"),
    "rb",
) as f:
    dataset = pkl.load(f)

In [ ]:
X_train, Y_train, y_train = dataset["X_train"], dataset["Y_train"], dataset["y_train"]
X_test, Y_test, y_test = dataset["X_test"], dataset["Y_test"], dataset["y_test"]
classes = Y_train.shape[-1]
print(X_train.shape)

In [ ]:
counts = np.unique(y_train, return_counts=True)[1]
n_samples = np.min(counts)
REPLACE = False

indexes = []
for ii in range(classes):
    class_idx = np.where(y_train == ii)[0]
    indexes.append(np.random.choice(class_idx, size=n_samples, replace=REPLACE))
indexes = np.concatenate(indexes)

X_train, Y_train = X_train[indexes], Y_train[indexes]

In [ ]:
BATCH_SIZE = 500
for i in range(0, X_train.shape[0], BATCH_SIZE):
    print(i)
    X_train[i : i + BATCH_SIZE], Y_train[i : i + BATCH_SIZE] = preprocess_batches(
        X_train[i : i + BATCH_SIZE], Y_train[i : i + BATCH_SIZE]
    )

In [ ]:
AUGDATA = True

if AUGDATA:
    augdata = data_augmentation(X_train, Y_train)
    X_train, Y_train = augdata.rotation(nrot=[0, 2], perc=1.0)

print(X_train.shape)

### Visualize

In [ ]:
for _ in range(5):
    fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(12, 4))
    ii = np.random.choice(np.arange(0, X_train.shape[0], 1, dtype=int))
    ax1.imshow(adjust_rgb(X_train[ii], 2, 98))
    ax2.imshow(Y_train[ii, :, :, 0])
    ax3.imshow(Y_train[ii, :, :, 1])
    ax4.imshow(Y_train[ii, :, :, 2])
    ax1.axis("off")
    ax2.axis("off")
    ax3.axis("off")
    ax4.axis("off")
    plt.show()

In [ ]:
dim = X_train.shape[1:]
proportions = np.sum(
    Y_train.reshape(
        Y_train.shape[0] * Y_train.shape[1] * Y_train.shape[2], Y_train.shape[-1]
    ),
    0,
)
total_pixels = Y_train.size
wmatrix = np.array(
    np.kron(total_pixels / proportions, np.ones((1, dim[0], dim[1], 1))),
    dtype=np.float32,
)
wmatrix = 0.0

### Defining the model

In [ ]:
BACKBONE = "efficientnetb7"
preprocess_input = sm.get_preprocessing(BACKBONE)

# Preprocess input.
X_train = preprocess_input(X_train)
X_test = preprocess_input(X_test)

In [ ]:
LOAD_WEIGHTS = True
loss = masked_loss(dim, ths=0.5, hold_out=0.1)

if not LOAD_WEIGHTS:
    # Define model.
    model = sm.Linknet(
        BACKBONE,
        classes=classes,
        activation="softmax",
        encoder_weights="imagenet",
        encoder_freeze=False,
    )

    # Learning rate.
    LR = 1.5e-5
    optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    model.compile(
        optimizer=optimizer,
        loss=loss.contrastive_loss,
        metrics=["acc"],
    )
# model.summary()

In [ ]:
today = str(date.today()).replace("-", "_")

CHECKPOINT_DIR = join("data", "models", "save_models")
CHECKPOINT_FILENAME = f"linket_{BACKBONE}_weights_{today}.h5"
checkpointer = callbacks.ModelCheckpoint(
    filepath=join(CHECKPOINT_DIR, CHECKPOINT_FILENAME),
    monitor="loss",
    verbose=1,
    save_best_only=True,
    mode="min",
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=10e-4,
    patience=50,
)

In [ ]:
if LOAD_WEIGHTS:
    CHECKPOINT_FILENAME = "linknet_efficientnetb7_weights_2023_07_05.h5"
    LR = 1.5e-5
    model = tf.keras.models.load_model(
        join(CHECKPOINT_DIR, CHECKPOINT_FILENAME), compile=False
    )
    loss = masked_loss(dim, ths=0.5, hold_out=0.1)
    optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    model.compile(
        optimizer,
        loss=loss.contrastive_loss,
        metrics=["acc"],
    )

In [ ]:
TRAINING = False
batch_size = 16
if TRAINING:
    history = model.fit(
        X_train,
        Y_train,
        batch_size=batch_size,
        validation_data=(X_test, Y_test),
        callbacks=[checkpointer, early_stopping],
        epochs=250,
    )

In [ ]:
if TRAINING:
    plt.plot(history.history["loss"])
    plt.plot(history.history["val_loss"])
    plt.title("Loss")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.legend(["train", "test"], loc="upper left")
    plt.savefig(
        join("data", "plots", f"graph_losses_{today}.png"), dpi=300, bbox_inches="tight"
    )
    plt.show()

### Visualize predictions

In [ ]:
b, e = 600, 610
pred_probs = model.predict(X_train[b:e])

n = 0
for i in range(b, e):
    fig, axs = plt.subplots(1, 4, figsize=(15, 6))

    axs[0].imshow(adjust_rgb(X_train[i], 10, 90))
    axs[0].axis("off")
    for i in range(3):
        axs[i + 1].imshow(pred_probs[n, :, :, i], cmap="plasma", vmin=0, vmax=1)
        axs[i + 1].axis("off")
    n += 1

In [ ]:
b, e = 60, 70
pred_probs = model.predict(X_test[b:e])

n = 0
for i in range(b, e):
    fig, axs = plt.subplots(1, 5, figsize=(15, 6))

    axs[0].imshow(adjust_rgb(X_test[i], 10, 90))
    axs[0].axis("off")
    axs[1].imshow(Y_test[i, :, :, 1], cmap="plasma", vmin=0, vmax=1)
    axs[1].axis("off")
    for i in range(3):
        axs[i + 2].imshow(pred_probs[n, :, :, i], cmap="plasma", vmin=0, vmax=1)
        axs[i + 2].axis("off")
    n += 1

In [ ]:
FOLDER_PATH = join("data", "images")
image_list = []

# Parcours de tous les fichiers du dossier.
for filename in os.listdir(FOLDER_PATH):
    if filename.endswith(".JPG") or filename.endswith(".jpeg"):
        # Chargement de l'image et ajout à la liste.
        img_path = join(FOLDER_PATH, filename)
        img = Image.open(img_path)
        img = ImageOps.exif_transpose(img)
        image_list.append(np.array(img))
        plt.imshow(img)
        plt.show()

## Applying the model to the set of images


In [ ]:
ii = np.random.choice(len(image_list), size=1)[0]
image, _ = undersample(image_list[ii], undersample_by=1)
dim = X_train.shape[1:]
XX = np.float32(
    cv2.bilateralFilter(np.float32(image), d=5, sigmaColor=35, sigmaSpace=35)
)
XX = preprocess_input(XX)
median_pixel_value = np.median(image[:100, :100])
imy, imx = np.where(image == median_pixel_value)[:2]
XX[imy, imx] = 0.0

In [ ]:
pred_tile = predict_tiles(model, merge_func=np.max, reflect=True)
pred_tile.create_batches(XX, (dim[0], dim[1], 3), step=int(dim[0]), n_classes=classes)
pred_tile.predict(batches_num=1500, coords_channels=False)
result = pred_tile.merge()
result[imy, imx] = 0.0

In [ ]:
fig, ax = plt.subplots(figsize=(25, 10))
ax.imshow(adjust_rgb(XX, 5, 99))
ax.imshow(result[:, :, 1], cmap="viridis", vmin=0.0, vmax=1.0, alpha=0.0)
plt.show()

In [ ]:
y = np.arange(result.shape[0])
x = np.arange(result.shape[1])
x, y = np.meshgrid(x, y)

In [ ]:
for c in range(classes):
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.imshow(adjust_rgb(XX, 5, 99), zorder=0)
    ax.pcolormesh(
        x,
        y,
        np.where(result[:, :, c] > 0.9, 1.0, np.nan),
        cmap="plasma",
        vmin=0.3,
        vmax=1.0,
        alpha=0.7,
        zorder=1,
    )
    plt.xlim(70, 2300)
    plt.ylim(200, 1800)
    plt.axis("off")
    plt.savefig(f"plots/pred_{c}.png", dpi=300, bbox_inches="tight")
    plt.show()